In [ ]:
import argparse

parser = argparse.ArgumentParser(description='Arguments for Feature Group creation.')

parser.add_argument('--version', default=1, required=False)

args = parser.parse_args()

In [ ]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store()
feature_view= fs.get_feature_view("transactions_view", args.version)

In [ ]:
X_train, y_train, X_test, y_test = feature_view.train_test_split(0.2)

## 🧬 Train Model

Next you'll train a model. Here, you set the class weight of the positive class to be twice as big as the negative class.


In [ ]:
from sklearn.linear_model import LogisticRegression

pos_class_weight = 0.9
clf = LogisticRegression(class_weight={0: 1.0 - pos_class_weight, 1: pos_class_weight}, solver='liblinear')
clf.fit(X_train, y_train.values.ravel())

Let's see how well it performs on your test data.

In [ ]:
from sklearn.metrics import classification_report
import pprint

preds = clf.predict(X_test)

report_dict = classification_report(y_test, preds, output_dict=True)
pprint.pprint(report_dict, width=2)

Pickle the model and write it to a local folder.

In [ ]:
import joblib

joblib.dump(clf, 'model.pkl')

In [ ]:
mr = project.get_model_registry()

## 📝 Export Model

Next you will export the model and attach additional information like the signature of the inputs/predictions, a concrete input example and evaluation metrics. In the last line, you upload the pickled model file to the Model Registry.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_test)
output_schema = Schema(y_test)

fraud_model = mr.sklearn.create_model("fraud",
                                      metrics={'accuracy': report_dict['accuracy']},
                                      input_example=X_test.head(1), 
                                      model_schema=ModelSchema(input_schema=input_schema, output_schema=output_schema))

fraud_model.save('model.pkl')